Ratatui is a crate for building terminal user interfaces in Rust.

In [2]:
//| code-fold: true

fn show_html<D>(content: D) where D: std::fmt::Display {
    println!(r#"EVCXR_BEGIN_CONTENT text/html
<div style="display: flex; justify-content:start; gap: 1em; margin: 1em">
{}
</div>
EVCXR_END_CONTENT"#, content);
}


In [4]:
//| code-fold: true
:dep ratatui = "0.26.2"
:dep ratatui-macros = "0.4.0"

## Layout primitives

We already saw that `Rect` is one of the primitives for rendering a widget.

We can create a `Rect` using `Rect::new(x, y, width, height)`:

In [7]:
use ratatui::widgets::Widget;

let (x, y, width, height) = (0, 0, 50, 5); 
let area = ratatui::layout::Rect::new(x, y, width, height);
let mut buf = ratatui::buffer::Buffer::empty(area);
ratatui::widgets::Block::bordered().render(area, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌────────────────────────────────────────────────┐",
        "│                                                │",
        "│                                                │",
        "│                                                │",
        "└────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

You can create inner `Rect`s by using the `Rect::inner` method:

In [10]:
let mut buf = ratatui::buffer::Buffer::empty(area);

let (horizontal, vertical) = (5, 1);
let inner_area = area.inner(&ratatui::layout::Margin::new(horizontal, vertical));

ratatui::widgets::Block::bordered().render(inner_area, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "                                                  ",
        "     ┌──────────────────────────────────────┐     ",
        "     │                                      │     ",
        "     └──────────────────────────────────────┘     ",
        "                                                  ",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Ratatui also has a layout solver using [the cassowary algorithm](https://en.wikipedia.org/wiki/Cassowary_(software)?useskin=vector).



In [11]:
use ratatui::layout::{Layout, Constraint};

let [first, second] = ratatui::layout::Layout::horizontal([Constraint::Length(10), Constraint::Length(10)]).areas(area);

let mut buf = ratatui::buffer::Buffer::empty(area);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌first───┐┌second──┐                              ",
        "│        ││        │                              ",
        "│        ││        │                              ",
        "│        ││        │                              ",
        "└────────┘└────────┘                              ",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

`ratatui-macros` has a macro to make some of this boilerplate simpler.

In [13]:
use ratatui_macros::{horizontal, vertical};

let (x, y, width, height) = (0, 0, 50, 6); 
let area = ratatui::layout::Rect::new(x, y, width, height);

let mut buf = ratatui::buffer::Buffer::empty(area);

let [top, middle, bottom] = vertical![*=1, *=1, *=1].areas(area);

let [first, second] = horizontal![==10, ==10].areas(top);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);

let [first, second] = horizontal![==10, ==10].flex(ratatui::layout::Flex::Center).areas(middle);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);

let [first, second] = horizontal![==10, ==10].flex(ratatui::layout::Flex::End).areas(bottom);
ratatui::widgets::Block::bordered().title("first").render(first, &mut buf);
ratatui::widgets::Block::bordered().title("second").render(second, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 6 },
    content: [
        "┌first───┐┌second──┐                              ",
        "└────────┘└────────┘                              ",
        "               ┌first───┐┌second──┐               ",
        "               └────────┘└────────┘               ",
        "                              ┌first───┐┌second──┐",
        "                              └────────┘└────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

In Ratatui, there are 3 fundamental text primitives that you should be aware of.

### Span

The first is a `Span`.

In [43]:
use ratatui::text::Span;

let span = Span::raw("hello world");
span

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }

A `Span` contains two fields. 

In [44]:
span.content

"hello world"

In [45]:
span.style

Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE }

A `Style` object contains foreground color, background color, and modifiers for whether the style being applied is **bold**, _italics_, etc

There are a number of constructors for `Span` that you may use, but `ratatui` exposes a `Stylize` trait that makes it easy to style content which I find very useful.

In [46]:
use ratatui::style::Stylize; // required trait to use style methods

"hello world".bold()

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: BOLD, sub_modifier: NONE } }

You can even chain these trait methods to add more styles:

In [47]:
"hello world".bold().yellow().on_black()

Span { content: "hello world", style: Style { fg: Some(Yellow), bg: Some(Black), underline_color: None, add_modifier: BOLD, sub_modifier: NONE } }

In [48]:
//| code-fold: true
fn show_span(s: Span) {
    let mut html = String::new();
    html.push_str("<span style=\"");

    // Set foreground color
    if let Some(color) = &s.style.fg {
        html.push_str(&format!("color: {};", color));
    }

    // Set background color
    if let Some(color) = &s.style.bg {
        html.push_str(&format!("background-color: {};", color));
    }

    // Add modifiers
    match s.style.add_modifier {
        ratatui::style::Modifier::BOLD => html.push_str("font-weight: bold;"),
        ratatui::style::Modifier::UNDERLINED => html.push_str("text-decoration: underline;"),
        _ => {}
    }
    html.push_str("\">");
    html.push_str(&s.content);
    html.push_str("</span>");
    show_html(html)
}

In [49]:
show_span("hello world".bold())

()

In [50]:
show_span("hello world".yellow().bold().on_black())

()

With `ratatui-macros`, you can even use a `format!` style macro to create a `Span`

In [51]:
use ratatui_macros::span;

let world = "world";
span!("hello {}", world)

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }

### Line

The second primitive to be aware of is a `Line`.

A line consists of one or more spans.

In [52]:
use ratatui::text::Line;

let line = Line::raw("hello world");
line

Line { spans: [Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }], style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE }, alignment: None }

In [53]:
line.spans

[Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }]

In [54]:
line.spans.len()

1

A unique feature of lines is that new lines are removed but the content is split into multiple spans. 

In [55]:
let line = Line::raw("hello world\ngoodbye world");

In [56]:
line.spans[0]

Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }

In [57]:
line.spans[1]

Span { content: "goodbye world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }

A line can also be styled with methods from the `Stylize` trait:

In [58]:
Line::raw("hello world").bold()

Line { spans: [Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }], style: Style { fg: None, bg: None, underline_color: None, add_modifier: BOLD, sub_modifier: NONE }, alignment: None }

In this case, the individual span's styles are left untouched but the `Line`'s style is updated.

Another unique feature about `Line` is that they can be aligned.

In [59]:
let centered_line = line.centered();
centered_line

Line { spans: [Span { content: "hello world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }, Span { content: "goodbye world", style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE } }], style: Style { fg: None, bg: None, underline_color: None, add_modifier: NONE, sub_modifier: NONE }, alignment: Some(Center) }

With `ratatui-macros`, you can create a `Line` using the `line!` macro using a `vec!`-like syntax.

In [60]:
use ratatui_macros::line;

line!["hello", " ", "world"].yellow().bold().centered();

Every element in the `line!` macro is converted to a `Span`.

### Text

Finally there is `Text`, which is a collection of `Line`s.

In [61]:
use ratatui::text::Text;

Text::from(vec![Line::raw("hello world"), Line::raw("goodbye world")]);

With ratatui-macros, you can create a `Text` using `text!` macro using a `vec!`-like syntax.

In [62]:
use ratatui_macros::text;

text!["hello world", "goodbye world"];

Here, every element in the `text!` macro is converted to a `Line`.

Like `Line`, `Text` can also be aligned. In this case, the alignment occurs on every `Line` inside the `Text`.

In [63]:
let t = text!["hello world", "goodbye world"].right_aligned();
t.alignment

Some(Right)

### Block

The simplest widget is the `Block` widget, which is essentially just borders.

In [64]:
let (x, y, width, height) = (0, 0, 50, 5); 
let area = Rect::new(x, y, width, height);
let mut buf = Buffer::empty(area);
Block::bordered().render(area, &mut buf);
buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌────────────────────────────────────────────────┐",
        "│                                                │",
        "│                                                │",
        "│                                                │",
        "└────────────────────────────────────────────────┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

Most widgets accept a `Block` as a fluent setter. We saw from earlier that the `Paragraph` has a `.block()` method that accepts a `Block`.

```rust
let paragraph = Paragraph::new(text).block(block).centered();
```

Blocks can have different kinds of borders:

In [65]:


let (x, y, width, height) = (0, 0, 50, 5); 
let area = Rect::new(x, y, width, height);
let mut buf = Buffer::empty(area);

Block::bordered().border_type(ratatui::widgets::BorderType::Double).render(area.inner(&ratatui::layout::Margin::new(2, 1)), &mut buf);
Block::bordered().borders(ratatui::widgets::Borders::TOP | ratatui::widgets::Borders::BOTTOM).render(area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "──────────────────────────────────────────────────",
        "  ╔════════════════════════════════════════════╗  ",
        "  ║                                            ║  ",
        "  ╚════════════════════════════════════════════╝  ",
        "──────────────────────────────────────────────────",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

And `Block` can have multiple titles in different locations:

In [66]:
let (x, y, width, height) = (0, 0, 50, 5); 
let area = Rect::new(x, y, width, height);
let mut buf = Buffer::empty(area);

let block = Block::bordered()
                .title("Top Left") // accepts anything that can be converted to a `Title` or a `Line`
                .title(Line::from("Top Center").centered()) // explicitly need to use `Line` if you want alignment
                .title(line!["Top Right"].right_aligned()) // you can use the `line!` macro to make it shorter
                .title(ratatui::widgets::block::Title::from("Bottom Right") // explicitly using `Title` gives you most control
                       .alignment(ratatui::layout::Alignment::Right)
                       .position(ratatui::widgets::block::title::Position::Bottom)
                )
                .title_bottom(Line::from("Bottom Center").centered()) // shorthand functions for bottom position
                .title_bottom("Bottom Left"); // aligned to the left by default

block.render(area, &mut buf);

buf

Buffer {
    area: Rect { x: 0, y: 0, width: 50, height: 5 },
    content: [
        "┌Top Left───────────Top Center──────────Top Right┐",
        "│                                                │",
        "│                                                │",
        "│                                                │",
        "└Bottom Left──────Bottom Center──────Bottom Right┘",
    ],
    styles: [
        x: 0, y: 0, fg: Reset, bg: Reset, underline: Reset, modifier: NONE,
    ]
}

## Conclusion

In the next post, we'll examine how text primitives work.